In [ ]:
import glob
import os
import pprint
import random
import shutil
from itertools import islice

import numpy as np
import pandas as pd
from pypcd import pypcd

In [ ]:
def convert_to_npy(path):
    cloud = pypcd.PointCloud.from_path(path)
    df = pd.DataFrame(cloud.pc_data)
    result = df[["x", "y", "z", "label"]].to_numpy()
    return result

In [ ]:
dataset_path = "/home/threedee/datasets/SmartLab/"
annotated_path = "/home/threedee/annotation_workspace/Data/smartlab/labled/"

senarios = ["senario1", "senario2", "senario3", "senario4"]

npy_output_path = dataset_path + "npy/"

In [ ]:
# save *.pcd files as *.npy files in given folder.

for sc in senarios:

    print("Processing files in: " + sc)
    anno_path_sc = annotated_path + sc + "/"
    save_path_sc = npy_output_path + sc + "/"

    list = glob.glob(anno_path_sc + "*.pcd")
    for i in range(len(list)):

        print(list[i])

        path, file = os.path.split(list[i])
        save_path = save_path_sc + file[:-4] + ".npy"

        if not os.path.exists(save_path_sc):
            os.makedirs(save_path_sc)

        npy_cloud = convert_to_npy(list[i])
        np.save(save_path, npy_cloud)

In [ ]:
# clean the train,val and test folders for new data

train_set = glob.glob(dataset_path + "train/*")
val_set = glob.glob(dataset_path + "val/*")
test_set = glob.glob(dataset_path + "test/*")
all_set = train_set + val_set + test_set
for f in all_set:
    # print("Removing: " + f)
    os.remove(f)
print("Removed " + str(len(all_set)) + " files.")

In [ ]:
# copy files in to train,val and test folders with random sampling

total = 218
split_val = [152, 44, 22]
split_names = ["train", "val", "test"]

files = glob.glob(npy_output_path + "/**/*.npy", recursive=True)

print(str(len(files)) + " files found.")
# pprint.pprint(files[0])

filearr = np.array(files)
np.random.shuffle(filearr)

# pprint.pprint(filearr[0])

splits = np.split(filearr, [152, 196])

for (idx, name) in zip(range(3), split_names):

    print("Copying " + str(len(splits[idx])) + " files into split: " + name)

    for file in splits[idx]:
        filename = os.path.split(file)[1]
        copy_path = dataset_path + name + "/" + filename

        shutil.copy(file, copy_path)